# Demo for HiPSter Inference using Illustris TNG

## Generate the HiPS tiles

The HiPS tiles are generated by reconstructing the data with the HiPS position on the spherical
latent space.

In [1]:
import hipster
import numpy as np


class Foo:
    def __init__(self):
        self.figsize_in_pixel = 128

    def __call__(self, data):
        data = np.clip(data.transpose(1, 2, 0), 0, 1) * 255
        return data


hipster.HiPSGenerator(
    decoder=hipster.Inference("data/illustris/models/full_trained/decoder.onnx"),
    image_maker=Foo(),
    max_order=0,
    hierarchy=2,
    hips_path="output/hips-illustris",
).execute()

Executing task: HiPSGenerator


## Catalog of training dataset



In [2]:
hipster.VOTableGenerator(
    encoder=hipster.Inference("data/illustris/models/encoder.onnx"),
    data_directory="data/illustris/parquet",
    output_file="output/illustris.vot",
).execute()

Executing task: VOTableGenerator


## Visualize HiPS tiles and catalog using Aladin

- [Aladin-Lite](https://github.com/cds-astro/aladin-lite)
- [PyAladin](https://github.com/cds-astro/ipyaladin)

In [3]:
from ipyaladin import Aladin
from ipywidgets import Layout

aladin = Aladin(survey="output/hips-illustris", fov=180, show_fullscreen_control=False)
aladin.add_catalog_from_URL("output/illustris.vot", {"source_size": 5, "color": "red"})
aladin

Aladin(survey='output/hips-illustris')

## Command Line Interface (CLI)

In production, the HiPS tiles are generated using the CLI, similar to the PyTorch Lightning CLI.
All configurations are written in a [yaml-file](./configs/hipster/illustris.yaml) and executed with the following command:

In [ ]:
!hipster --config configs/hipster/illustris.yaml

Base URL: http://localhost:8083
Tasks:
  - HiPSGenerator
  - VOTableGenerator
Executing task: HiPSGenerator
Executing task: VOTableGenerator
Generating HTML page...


Now, let's run a webserver to visualize Aldin-Lite with the produced HiPS tiles and catalog:

In [9]:
!python -m http.server 8083 --bind localhost --directory HiPSter

Serving HTTP on 127.0.0.1 port 8083 (http://127.0.0.1:8083/) ...
127.0.0.1 - - [23/Apr/2025 10:40:07] "GET /illustris/model/properties HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2025 10:40:07] "GET /illustris.vot HTTP/1.1" 200 -

Keyboard interrupt received, exiting.
^C
